#### Import the Packages

In [1]:
import pandas as pd
import numpy as np

#### Importing the data

In [2]:
raw_csv_data=pd.read_csv("Index2018.csv")
df_comp=raw_csv_data.copy()

#### Length of the Time Period

We want to transform the "date" column into a "date type". 
We want to start by checking the counts.

In [3]:
df_comp.describe()

,spx,dax,ftse,nikkei
count,6269.000000,6269.000000,6269.000000,6269.000000
mean,1288.127542,6080.063363,5422.713545,14597.055700
std,487.586473,2754.361032,1145.572428,4043.122953
min,438.920000,1911.700000,2876.600000,7054.980000
25%,990.671905,4069.350000,4486.100000,10709.290000
50%,1233.420000,5773.340000,5662.430000,15028.170000
75%,1459.987747,7443.070000,6304.250000,17860.470000
max,2872.867839,13559.600000,7778.637689,24124.150000


Python does not include date in the describe method result because it is simply text representation to python.
One way to work around is to specify the column.

In [4]:
df_comp.date.describe()

count           6269
unique          6269
top       23/02/2012
freq               1
Name: date, dtype: object

However, this information is very limited. The top value here is actually arbitrary. Becaues all the "date" value are 1. Any single data holds a "top" value. Thus, the method will randomly selects one to display. Thus, we need to convert date into date type in order to analyze.

#### From Text to Date

In [5]:
df_comp.head()

,date,spx,dax,ftse,nikkei
0,07/01/1994,469.90,2224.95,3445.98,18124.01
1,10/01/1994,475.27,2225.00,3440.58,18443.44
2,11/01/1994,474.13,2228.10,3413.77,18485.25
3,12/01/1994,474.17,2182.06,3372.02,18793.88
4,13/01/1994,472.47,2142.37,3360.01,18577.26


In [6]:
df_comp.date=pd.to_datetime(df_comp.date,dayfirst=True) #need to override

to_datetime method assumes we are plugging in a string in a "mm/dd/yyyy" form. but our data is "dd/mm/yyyy" format so we need the optional argument dayfirst=True.



In [7]:
df_comp.head()

,date,spx,dax,ftse,nikkei
0,1994-01-07,469.90,2224.95,3445.98,18124.01
1,1994-01-10,475.27,2225.00,3440.58,18443.44
2,1994-01-11,474.13,2228.10,3413.77,18485.25
3,1994-01-12,474.17,2182.06,3372.02,18793.88
4,1994-01-13,472.47,2142.37,3360.01,18577.26


In [8]:
#Check by describing the date column
df_comp.date.describe()

count                    6269
unique                   6269
top       1999-08-23 00:00:00
freq                        1
first     1994-01-07 00:00:00
last      2018-01-29 00:00:00
Name: date, dtype: object

Now it starts at 1994-01-07 00:00:00 and ends 1999-08-23 00:00:00. Note pandas datetime type has hour/minutes/seconds as well.

Each value should correspond to a time period. We often want to examine specific chunks of data between two concrete dates. Thus, we need to use the associated time periods as indices。 In this way, all the other columns would automatically becomes times series, since there is a time period attached to each value.

#### Setting the Index

In [9]:
df_comp.set_index("date",inplace=True)

In [10]:
df_comp.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


In [16]:
df_comp.date.describe()

AttributeError: 'DataFrame' object has no attribute 'date'

Because we turned date value as index, it is not saved as a separate attribute in the data frame. The index is relatively fixed, so we need to make sure they are correct before converting.

Because time series data requires constant time-series frequency throughout the set, we need to define the frequency.

#### Setting the desired frequency

In [11]:
df_comp=df_comp.asfreq('d')

'h' - hourly
'w' - weekly
'd' - daily
'm' - monthly
'a' - annual 

In [12]:
df_comp.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-08,NaN,NaN,NaN,NaN
1994-01-09,NaN,NaN,NaN,NaN
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25


We can see we generated new periods, which do not have values attached. Because our data is financial prices, we only have working days.

In [13]:
df_comp=df_comp.asfreq('b')

'b' represents 'business days'

In [21]:
df_comp.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


Now we don't have those missing days.

#### Handling Missing Values

In [14]:
df_comp.isna()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,False,False,False,False
1994-01-10,False,False,False,False
1994-01-11,False,False,False,False
1994-01-12,False,False,False,False
1994-01-13,False,False,False,False
...,...,...,...,...
2018-01-23,False,False,False,False
2018-01-24,False,False,False,False
2018-01-25,False,False,False,False


True ---missing values for the period

False ---- no missing values for the period

Since True=1, Fales=0, we can get summation to get total number of missing values.

In [15]:
df_comp.isna().sum()

spx       8
dax       8
ftse      8
nikkei    8
dtype: int64

Initially, we don't have missing values so we must have created missing values by changing the frequency to "b". We need to fill those missing values with some values.

"fillna()" method:

1. Front filling: Asssigns the value of the previous period
2. Back filling: Assigns the value of the next period
3. Assigning the same value: Assign the average to all the missing values within the time series ---- it is usually  a bad practice because there are underlying time patterns to the data. It only works when the data heavily fluctuates around the mean from the first to last day.

We can try differnt methods.

In [17]:
df_comp.spx=df_comp.spx.fillna(method="ffill") #forward/front filling

In [18]:
df_comp.isna().sum() #check the number of missing values of spx

spx       0
dax       8
ftse      8
nikkei    8
dtype: int64

It works as expected.

In [19]:
df_comp.ftse=df_comp.ftse.fillna(method="bfill")

In [20]:
df_comp.isna().sum() #check

spx       0
dax       8
ftse      0
nikkei    8
dtype: int64

In [21]:
df_comp.dax=df_comp.dax.fillna(value=df_comp.dax.mean()) #assign the same value does not need the method argument but needs to set the value argument

In [22]:
df_comp.isna().sum()

spx       0
dax       0
ftse      0
nikkei    8
dtype: int64

#### Simplifying the Dataset

Now we would like to analyze only how the S&P 500 performs. we can remove the columns for the FTSE, DAX and NIKKEI. 

1) The less data we load, the faster we can manipulate it.
2) Clarity. Easier to keep track of the dataset.

We want to create a new column so we can change the values by chaning only one line of the code.

In [23]:
df_comp['market_value']=df_comp.spx #Because "market_value" is a column name, we use []

In [24]:
df_comp.describe()

,spx,dax,ftse,nikkei,market_value
count,6277.000000,6277.000000,6277.000000,6269.000000,6277.000000
mean,1288.642547,6080.063363,5423.690398,14597.055700,1288.642547
std,487.868210,2752.604984,1145.568370,4043.122953,487.868210
min,438.920000,1911.700000,2876.600000,7054.980000,438.920000
25%,992.715221,4070.460000,4487.880000,10709.290000,992.715221
50%,1233.761241,5774.380000,5663.300000,15028.170000,1233.761241
75%,1460.250000,7442.660000,6304.630175,17860.470000,1460.250000
max,2872.867839,13559.600000,7778.637689,24124.150000,2872.867839


In [25]:
del df_comp['spx'] #remove

In [26]:
df_comp.describe()

,dax,ftse,nikkei,market_value
count,6277.000000,6277.000000,6269.000000,6277.000000
mean,6080.063363,5423.690398,14597.055700,1288.642547
std,2752.604984,1145.568370,4043.122953,487.868210
min,1911.700000,2876.600000,7054.980000,438.920000
25%,4070.460000,4487.880000,10709.290000,992.715221
50%,5774.380000,5663.300000,15028.170000,1233.761241
75%,7442.660000,6304.630175,17860.470000,1460.250000
max,13559.600000,7778.637689,24124.150000,2872.867839


In [ ]:
del df_comp['dax'],df_comp['ftse']df_comp['nikkei'] #removing at once

In [30]:
df_comp.describe()

,nikkei,market_value
count,6269.000000,6277.000000
mean,14597.055700,1288.642547
std,4043.122953,487.868210
min,7054.980000,438.920000
25%,10709.290000,992.715221
50%,15028.170000,1233.761241
75%,17860.470000,1460.250000
max,24124.150000,2872.867839


### Splitting the data

We need to split the data into training/ test set to do machine learning. Usually, we "shuffle" the data before splittig. However, time series data relies on keeping the chronological order of the values. Thus, shuffling is not possible. 

Training set: From the beginning up to some cut off point.

Testing set: From the cut off point until the end.

For this course, we will use 80% vs 20% split.

iloc() method (the name "iloc" comes from the index location) will be used to slice the data.

In [35]:
size=int(len(df_comp)*0.8) #int method assures the 0.8 cutoff point is a point

In [36]:
df=df_comp.iloc[:size] #it should be df_train but we simplify the name here for the purpose of the course

In [37]:
df_test=df_comp.iloc[size:]

To check whether these two sets have overlaps:

In [38]:
df.tail()

,market_value
date,
2013-04-01,1562.173837
2013-04-02,1570.252238
2013-04-03,1553.686978
2013-04-04,1559.979316
2013-04-05,1553.278930


In [39]:
df_test.head()

,market_value
date,
2013-04-08,1563.071269
2013-04-09,1568.607909
2013-04-10,1587.731827
2013-04-11,1593.369863
2013-04-12,1588.854623
